# Building AI Agents with Persistent Memory using Cognee

This notebook demonstrates how to create intelligent AI agents with advanced memory capabilities using [**cognee**](https://www.cognee.ai/) - an open-source AI memory system that integrates knowledge graphs, semantic search, and session management to develop context-aware AI systems.

## 🎯 Learning Objectives

By the end of this tutorial, you will learn how to:
- **Build Knowledge Graphs Backed by Embeddings**: Convert unstructured text into structured, queryable knowledge
- **Implement Session Memory**: Enable multi-turn conversations with automatic context retention
- **Persist Conversations**: Optionally save significant interactions in long-term memory for future use
- **Query Using Natural Language**: Retrieve and utilize historical context in new conversations
- **Visualize Memory**: Examine the connections within your agent's knowledge graph


## 🏗️ What You'll Build

In this tutorial, we will create a **Coding Assistant** with persistent memory that:

### 1. **Knowledge Base Construction**
   - Gathers information about developer profiles and expertise
   - Processes principles and best practices of Python programming
   - Stores historical conversations between developers and AI assistants

### 2. **Session-Aware Conversations**
   - Maintains context across multiple questions within the same session
   - Automatically saves each question/answer pair for efficient retrieval
   - Delivers coherent, context-aware responses based on conversation history

### 3. **Long-term Memory**
   - Saves important conversations into a long-term memory
   - Retrieves relevant memories from the knowledge base and past sessions to enhance new interactions
   - Builds an expanding knowledge base that improves over time

### 4. **Intelligent Memory Retrieval**
   - Utilizes graph-aware semantic search to locate relevant information across all stored knowledge
   - Filters searches by data categories (developer information vs. principles)
   - Combines multiple data sources to provide comprehensive answers


## 📋 Prerequisites & Setup

### System Requirements

Before you begin, make sure you have:

1. **Python Environment**
   - Python 3.9 or higher
   - Virtual environment (recommended)
   
2. **Redis Cache** (Required for Session Management)
   - Local Redis: `docker run -d -p 6379:6379 redis`
   - Or use a managed Redis service
   
3. **LLM API Access**
   - OpenAI API key or other providers (see [documentation](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Database Configuration**
   - No configuration required by default. Cognee uses file-based databases (LanceDB and Kuzu)
   - Optionally, you can set up Azure AI Search as a vector store (see [documentation](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Environment Configuration

Create a `.env` file in your project directory with the following variables:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Understanding Cognee's Memory Architecture

### How Cognee Works

Cognee offers an advanced memory system that surpasses basic key-value storage:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Key Components:

1. **Knowledge Graph**: Stores entities, relationships, and semantic connections
2. **Vector Embeddings**: Enables semantic search across all stored information
3. **Session Cache**: Keeps track of conversation context within and across sessions
4. **NodeSets**: Organize data into logical categories for focused retrieval

### Memory Types in This Tutorial:

- **Persistent Memory**: Long-term storage in the knowledge graph
- **Session Memory**: Temporary conversation context stored in Redis cache
- **Semantic Memory**: Vector-based similarity search across all data


## 📦 Install Required Packages

Install Cognee with Redis support for session management:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Initialize Environment and Load Libraries

Make sure:
1. Redis is running (e.g., via Docker: `docker run -d -p 6379:6379 redis`)
2. Environment variables are set before importing cache modules
3. If needed, restart the kernel and run cells in order

The following cell will:
1. Load environment variables from `.env`
2. Configure Cognee with your LLM settings
3. Enable caching for session management
4. Validate all components are properly connected


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Configure Storage Directories

Cognee uses two separate directories for its operations:
- **Data Root**: Stores ingested documents and processed data
- **System Root**: Contains the knowledge graph database and system metadata

We'll create isolated directories for this tutorial as follows:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Reset Memory State

Before we start building our memory system, let's make sure we're starting with a clean slate.

> 💡 **Tip**: You can skip this step if you want to keep existing memories from your previous runs when using this notebook later.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Part 1: Building the Knowledge Base

### Data Sources for Our Developer Assistant

We'll incorporate three types of data to build a comprehensive knowledge base:

1. **Developer Profile**: Personal expertise and technical background
2. **Python Best Practices**: The Zen of Python along with practical guidelines
3. **Historical Conversations**: Past Q&A sessions between developers and AI assistants

This diverse data enables our agent to:
- Understand the user's technical context
- Provide recommendations based on best practices
- Learn from previous successful interactions


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Process Data into Knowledge Graph

Now we'll transform our raw text into a structured memory. This process:

1. **Adds data to NodeSets**: Organizes information into logical categories
   - `developer_data`: Developer profile and conversations
   - `principles_data`: Python best practices and guidelines

2. **Runs Cognify Pipeline**: Extracts entities, relationships, and creates embeddings
   - Identifies key concepts
   - Creates semantic connections between related information
   - Generates vector embeddings

This may take a few moments as the LLM processes the text and builds the graph structure:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Visualize the Knowledge Graph

Let's examine the structure of our knowledge graph. The visualization includes:
- **Nodes**: Entities extracted from the text (concepts, technologies, individuals)
- **Edges**: Relationships and connections between entities
- **Clusters**: Related concepts grouped by semantic similarity

Open the generated HTML file in your browser to interactively explore the graph:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Enhance Memory with Memify

The `memify()` function examines the knowledge graph and formulates smart rules about the data. This process:
- Detects patterns and optimal practices
- Develops practical guidelines derived from the content
- Builds connections between various knowledge domains

These rules enable the agent to provide more insightful answers to questions. Capturing a second visualization allows you to observe how the graph becomes more interconnected after enrichment.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Part 2: Intelligent Memory Retrieval

### Demonstration 1: Cross-Document Knowledge Integration

Now that our knowledge graph is built, let's test how Cognee combines information from multiple sources to answer complex questions.

The first query demonstrates:
- **Semantic understanding**: Identifying relevant concepts even if they are not explicitly mentioned
- **Cross-referencing**: Merging developer profiles with Python principles
- **Contextual reasoning**: Applying best practices to specific implementations

### Demonstration 2: Filtered Search with NodeSets

The second query illustrates how to focus on specific subsets of the knowledge graph:
- Utilizes the `node_name` parameter to search exclusively within `principles_data`
- Delivers targeted answers from a particular knowledge domain
- Ideal for situations requiring domain-specific insights


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Part 3: Session Management Setup

### Enabling Conversation Memory

Session management is essential for preserving context throughout multiple interactions. Here we will:

1. **Initialize User Context**: Create or retrieve a user profile for session tracking
2. **Configure Cache Engine**: Connect to Redis to store conversation history
3. **Enable Session Variables**: Set up context variables that remain consistent across queries

> ⚠️ **Important**: This requires Redis to be running and `CACHING=true` in your environment


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Helper Function: View Session History

This utility function allows us to check the conversation history stored in Redis. It's useful for:
- Debugging session management
- Ensuring that conversations are being cached
- Understanding what context is available to the agent


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Session 1: Async Support Lab — First Question

Start the `async-support-lab` session by asking for asyncio patterns that are telemetry-friendly for a large-scale web scraper. The graph is already familiar with asyncio, aiohttp, and monitoring practices, so the response should reflect previous discussions while adapting the answer to this new question.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Inspect Session 1 Memory After the First Exchange

Running `show_history(session_1)` immediately after the initial question confirms that Cognee saved both the prompt and the completion into Redis. You should see one entry with the concurrency guidance.


In [ ]:
await show_history(session_1)

## Session 1: Follow-up on Data Models

Next, we ask, "When should I choose dataclasses versus Pydantic?" using the same session ID. Cognee should combine Python principles with previous FastAPI discussions to offer detailed advice—showing that context is maintained within a named session.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Confirm Session 1 History Contains Both Turns

Another `show_history(session_1)` call should reveal two Q&A entries. This corresponds to the "memory replay" step in the Mem0 lab and demonstrates that additional turns are added to the same transcript.


In [ ]:
await show_history(session_1)

## Session 2: Design Review Thread — Fresh Session

To demonstrate isolation between threads, we initiate `design-review-session` and request logging guidance for incident reviews. Although the underlying knowledge base remains the same, the new session ID ensures that transcripts are kept separate.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Review Session 2 History

`show_history(session_2)` should only display the design-review prompt/response pair. Compare it with Session 1 to emphasize how Cognee maintains separate transcripts while leveraging the shared knowledge graph.


In [ ]:
await show_history(session_2)

## Summary

Congratulations! You’ve just equipped your coding assistant with a robust long-term memory layer powered by Cognee.

In this tutorial, you transformed raw developer content (code, documentation, chats) into a graph + vector memory that your agent can search, reason about, and continuously enhance.

What You’ve Learned:

1. **From raw text to AI memory**: How Cognee processes unstructured data and converts it into intelligent, searchable memory using a combined vector + knowledge graph architecture.

2. **Graph enrichment with memify**: How to go beyond basic graph creation by using memify to add derived facts and richer relationships to your existing graph.

3. **Multiple search strategies**: How to query memory using various search methods (graph-aware Q&A, RAG-style completion, insights, raw chunks, code search, etc.) based on your agent’s needs.

4. **Visual exploration**: How to inspect and debug what Cognee has built using graph visualizations and the Cognee UI, allowing you to see how knowledge is structured.

5. **Session-aware memory**: How to integrate per-session context with persistent semantic memory, enabling agents to retain information across sessions without leaking data between users.


## Key Takeaways
1. Memory as a Knowledge Graph backed by Embeddings

    - **Structured understanding**: Cognee combines a vector store and a graph store, making your data both searchable by meaning and connected through relationships. By default, Cognee uses file-based databases (LanceDB for vector storage, Kuzu for graph database).

    - **Relationship-aware retrieval**: Answers are grounded not just in “similar text” but also in how entities are interconnected.

    - **Living memory**: The memory layer evolves, grows, and remains queryable as a unified graph.

2. Search & Reasoning Modes
    - **Hybrid retrieval**: Combines vector similarity, graph structure, and LLM reasoning, ranging from raw chunk lookups to graph-aware question answering.

    - **Adapt the mode to the task**: Use completion-style modes for natural language answers, and chunk/summary/graph modes when your agent needs raw context or to drive its own reasoning.

3. Personalized, Session-Aware Agents
    - **Session context + long-term memory**: Cognee separates short-term “thread” context from long-term, user- or organization-level memory.

## Real-World Applications

1. **Vertical AI Agents**

    Use the approach outlined in this notebook to create domain-specific copilots powered by Cognee as their retrieval and reasoning backbone:

- **Developer copilots**: Assist with code reviews, incident analysis, and architecture by navigating code, APIs, design documents, and tickets as a unified memory graph.

- **Customer-facing copilots**: Support or success agents that retrieve information from product documentation, FAQs, CRM notes, and past tickets using graph-aware retrieval and cited answers.

- **Internal expert copilots**: Policy, legal, or security assistants that reason over interconnected rules, guidelines, and historical decisions instead of isolated PDFs.

    Cognee is designed to serve as a persistent, accurate memory for AI agents, providing a living knowledge graph that integrates seamlessly with your agent, replacing ad-hoc combinations of vector stores and custom graph code.

2. **Unifying Data Silos into One Memory**

    This approach also enables the creation of a unified memory layer across disparate data sources:

- **From silos to one graph**: Combine structured (e.g., databases) and unstructured data (e.g., documents, chats) into a single graph backed by embeddings, rather than maintaining separate indices for each system.

- **Cross-source reasoning with citations**: Perform multi-step reasoning across all data—“join” logs, metrics, and documents via the graph—and still provide grounded answers with proper citations.

- **Knowledge hubs**: In domains like banking or education, Cognee is already being used to unify PDFs, internal systems, and app data into a single knowledge graph with vectors, enabling agents to answer questions with precise, cited context.

## Next Steps

You’ve implemented the core memory loop. Here are some natural extensions you can explore on your own (refer to [Cognee documentation](https://docs.cognee.ai/) for more details):

1. **Experiment with temporal awareness**: Enable temporal cognify to extract events and timestamps from text.

2. **Introduce ontology-driven reasoning**: Define an OWL ontology for your domain. Use Cognee’s ontology support to ground extracted entities and relationships in that schema, enhancing graph quality and domain-specific answers.

3. **Add a feedback loop**: Allow Cognee to adjust graph edge weights based on real user feedback, so retrieval improves over time instead of remaining static.

4. **Tune for personalization & session behavior**: Use user IDs, tenants, and datasets to provide each individual or team with their own view of the shared memory engine.

5. **Scale out to more complex agents**: Integrate Cognee into agent frameworks to build multi-agent systems that share the same memory layer. *Microsoft Agent Framework x Cognee plugin is coming soon.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
This document has been translated using the AI translation service [Co-op Translator](https://github.com/Azure/co-op-translator). While we aim for accuracy, please note that automated translations may include errors or inaccuracies. The original document in its native language should be regarded as the authoritative source. For critical information, professional human translation is advised. We are not responsible for any misunderstandings or misinterpretations resulting from the use of this translation.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
